# Game Title Detection

In [ ]:
!pip install kaggle timm

In [ ]:
import os
import zipfile
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import timm
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy

try:
    from google.colab import files
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

In [ ]:
# Configuration
RANDOM_STATE = 42
EPOCHS = 15
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_CLASSES = 5
IMAGE_SIZE = 224

## Data Loading Functions

In [ ]:
def download_kaggle_data():
    """Download and extract Kaggle competition data."""
    if COLAB_ENV:
        try:
            uploaded = files.upload()
        except Exception as e:
            print(f"File upload failed: {e}")
    else:
        print("Running outside of Colab. Ensure kaggle.json is in ~/.kaggle/")

    if 'kaggle.json' in os.listdir('.'):
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    else:
        print("kaggle.json not found.")

    if not os.path.exists('cpe342-karena.zip'):
        print("Downloading data...")
        !kaggle competitions download -c cpe342-karena
    else:
        print("Data already downloaded.")

    if os.path.exists('cpe342-karena.zip'):
        print("Extracting data...")
        with zipfile.ZipFile('cpe342-karena.zip', 'r') as zip_ref:
            zip_ref.extractall('.')
        print("Data extracted.")

## Data Preprocessing Functions

In [ ]:
def get_transforms():
    """Create training and validation transforms."""
    train_transform = T.Compose([
        T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        T.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
        T.RandomHorizontalFlip(),
        T.RandomRotation(15),
        T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
        T.ToTensor()
    ])
    
    val_transform = T.Compose([
        T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        T.ToTensor()
    ])
    
    return train_transform, val_transform

In [ ]:
class GameDataset(Dataset):
    """Dataset for game images."""
    def __init__(self, df, transform, data_dir, is_test=False):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        self.data_dir = data_dir
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.data_dir, row.file_name)
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        
        if self.is_test:
            return img, row.file_name
        return img, int(row.label)

In [ ]:
def create_data_loaders(train_df, val_df, test_df):
    """Create data loaders for training, validation, and testing."""
    train_transform, val_transform = get_transforms()
    
    train_ds = GameDataset(train_df, train_transform, "public_dataset/task4/train")
    val_ds = GameDataset(val_df, val_transform, "public_dataset/task4/train")
    test_ds = GameDataset(test_df, val_transform, "public_dataset/task4/test", is_test=True)
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    
    return train_loader, val_loader, test_loader

## Model Functions

In [ ]:
def create_model(device):
    """Create and configure the model."""
    model = timm.create_model("vit_base_patch16_224", pretrained=True, num_classes=NUM_CLASSES)
    model = model.to(device)
    
    mixup_fn = Mixup(mixup_alpha=0.2, cutmix_alpha=0.2, num_classes=NUM_CLASSES)
    criterion = SoftTargetCrossEntropy()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    return model, mixup_fn, criterion, optimizer, scheduler

In [ ]:
def train_epoch(model, train_loader, mixup_fn, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        imgs, labels = mixup_fn(imgs, labels)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss

In [ ]:
def validate_model(model, val_loader, device):
    """Validate the model and return F1 score."""
    model.eval()
    preds, trues = [], []
    
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds += outputs.argmax(1).cpu().numpy().tolist()
            trues += labels.numpy().tolist()
    
    return f1_score(trues, preds, average='macro')

## Test Time Augmentation Functions

In [ ]:
def get_tta_transforms():
    """Create TTA transforms."""
    return [
        T.Compose([T.Resize((IMAGE_SIZE, IMAGE_SIZE)), T.ToTensor()]),
        T.Compose([T.Resize((IMAGE_SIZE, IMAGE_SIZE)), T.RandomHorizontalFlip(p=1.0), T.ToTensor()]),
        T.Compose([lambda img: T.functional.rotate(img, 10), T.Resize((IMAGE_SIZE, IMAGE_SIZE)), T.ToTensor()]),
        T.Compose([lambda img: T.functional.rotate(img, -10), T.Resize((IMAGE_SIZE, IMAGE_SIZE)), T.ToTensor()]),
        T.Compose([T.CenterCrop(200), T.Resize((IMAGE_SIZE, IMAGE_SIZE)), T.ToTensor()]),
        T.Compose([T.Resize((IMAGE_SIZE, IMAGE_SIZE)), T.ColorJitter(brightness=0.3), T.ToTensor()]),
        T.Compose([T.Resize((IMAGE_SIZE, IMAGE_SIZE)), T.ColorJitter(contrast=0.3), T.ToTensor()])
    ]

In [ ]:
def predict_with_tta(model, test_loader, device):
    """Make predictions using TTA."""
    tta_transforms = get_tta_transforms()
    pred_list = []
    filenames = []
    
    with torch.no_grad():
        for imgs, names in test_loader:
            for name in names:
                img_path = f"public_dataset/task4/test/{name}"
                img = Image.open(img_path).convert("RGB")
                
                one_image_logits = []
                for t in tta_transforms:
                    aug_img = t(img).unsqueeze(0).to(device)
                    logits = model(aug_img).softmax(dim=1)
                    one_image_logits.append(logits)
                
                avg_logits = torch.mean(torch.stack(one_image_logits), dim=0)
                pred = avg_logits.argmax(1).cpu().numpy()[0]
                
                pred_list.append(pred)
                filenames.append(name)
    
    return pred_list, filenames

# Training Pipeline

In [ ]:
# Download data
download_kaggle_data()

In [ ]:
# Load and split data
train_df = pd.read_csv('public_dataset/task4/train.csv')
test_df = pd.read_csv('public_dataset/task4/test_refined.csv')
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=RANDOM_STATE)

In [ ]:
# Create data loaders
train_loader, val_loader, test_loader = create_data_loaders(train_df, val_df, test_df)

In [ ]:
# Initialize model and training components
device = "cuda" if torch.cuda.is_available() else "cpu"
model, mixup_fn, criterion, optimizer, scheduler = create_model(device)

## Model Training

In [ ]:
# Training loop
best_f1 = 0.0

for epoch in range(EPOCHS):
    total_loss = train_epoch(model, train_loader, mixup_fn, criterion, optimizer, device)
    scheduler.step()
    
    f1 = validate_model(model, val_loader, device)
    
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), "best_model.pth")
    
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss:.4f} - F1: {f1:.4f}")

print(f"Best F1: {best_f1:.4f}")

# Prediction Pipeline

In [ ]:
# Load best model
model = timm.create_model("vit_base_patch16_224", pretrained=False, num_classes=NUM_CLASSES)
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model = model.to(device)
model.eval()
print("Best model loaded successfully!")

In [ ]:
# Generate predictions with TTA
pred_list, filenames = predict_with_tta(model, test_loader, device)

# Create submission
submission = pd.DataFrame({
    "filename": filenames,
    "label": pred_list
})

submission.to_csv("submission.csv", index=False)
print(f"Submission created with {len(submission)} predictions")
print(f"Prediction distribution: {submission['label'].value_counts().to_dict()}")

if COLAB_ENV:
    files.download('submission.csv')